In [1]:

from Experiment import Experiment
import matplotlib.pyplot as plt
%matplotlib qt

loadir = 'H:/My Drive/dark 2022/csv_dark/' # dir to save the data frame in # H:\My Drive\Ronis Exp\sagiv\data_analysis
exp_name = ['manipulated_2023_08_10_100ms','manipulated_2022_02_03_dark']#['2022_02_03','2022_03_10','2022_05_19','2022_03_03'] # 22_11_28
ref_frame = 0

experiment = Experiment(loadir,exp_name[0])
experiment2 = Experiment(loadir,exp_name[1])




KeyboardInterrupt: 

In [ ]:

fig = experiment.plot_movies('time','CM_xy_dot', case = 'plot_exp', fig = False, mov = False)
fig = experiment2.plot_movies('time','CM_xy_dot', case = 'plot_exp', fig = fig, mov = False)

fig.show()
fig = experiment.plot_movies('time','pitch_y_frame', case = 'plot_exp', fig = False, mov = False)
fig = experiment2.plot_movies('time','pitch_y_frame', case = 'plot_exp', fig = fig, mov = False)


fig.show()


UnboundLocalError: local variable 'color' referenced before assignment

In [10]:
mov = [1,11,12,13,14,17,20,21,22,24,25,3,30,32,35,37,38,39,4,41,43,44,48,5,52,54,6,61,66,67,68,70,72,73,76,8]
mov_name = [f'mov{mov}' for mov in mov]

fig = experiment.plot_movies('time','CM_xy_dot', case = 'plot_mov', fig = False, mov = False)
fig.show()
fig = experiment.plot_movies('time','pitch_y_frame', case = 'plot_mov', fig = False, mov = False,prop = 'mean_body')

fig.show()

fig = experiment.plot_movies('CM_real_y_body','CM_real_x_body', case = 'plot_mov', fig = False, mov = False)
fig.show()

In [3]:
data = experiment.exp_dict['mov2'].data['body']



In [64]:
idx_frame = experiment.exp_dict['mov2'].header['body']['frames']
data['mov3'].data['body'][:,idx_frame]

array([ 169.,  170.,  171., ..., 4066., 4067., 4068.])

In [3]:
import numpy as np
mov = [1,11,12,13,14,17,20,21,22,24,25,3,30,32,35,37,38,39,4,41,43,44,48,5,52,54,6,61,66,67,68,70,72,73,76,8]
mov_name = [f'mov{mov}' for mov in mov]
idx = experiment.exp_dict['mov2'].header['body']['CM_xy_dot']
idx_time = experiment.exp_dict['mov2'].header['body']['time']
idx_vecolity = experiment.exp_dict['mov2'].header['body']['CM_xy_dot']
idx_pitch = experiment.exp_dict['mov2'].header['body']['pitch_body']


data = experiment.exp_dict

time_zero_v = np.hstack([data[mov_name].data['body'][np.where(np.diff(np.sign(data[mov_name].data['body'][:,idx]))<0)[0],idx_time] for mov_name in mov_name])
time_min_v = np.hstack([data[mov_name].data['body'][np.argmin(data[mov_name].data['body'][:,idx_vecolity]),idx_time] for mov_name in mov_name])
time_max_pitch = np.hstack([data[mov_name].data['body'][np.argmax(data[mov_name].data['body'][:,idx_pitch]),idx_time] for mov_name in mov_name])



In [55]:
data['mov2'].data.keys()

dict_keys(['body', 'mean_body', 'mean_wing', 'vectors', 'wing'])

In [7]:
plt.figure()
plt.hist(time_zero_v,bins = 15)
plt.figure()

plt.hist(time_min_v,bins = 15)
plt.figure()

plt.hist(time_max_pitch,bins = 15)

(array([1., 0., 0., 0., 4., 4., 6., 4., 8., 2., 4., 1., 1., 0., 1.]),
 array([  1.25      ,  12.13333333,  23.01666667,  33.9       ,
         44.78333333,  55.66666667,  66.55      ,  77.43333333,
         88.31666667,  99.2       , 110.08333333, 120.96666667,
        131.85      , 142.73333333, 153.61666667, 164.5       ]),
 <BarContainer object of 15 artists>)

In [4]:
experiment.exp_dict['mov1'].header['mean_body']

{'pitch_body': 0,
 'yaw_body': 1,
 'roll_body': 2,
 'pitch_body_dot': 3,
 'yaw_body_dot': 4,
 'roll_body_dot': 5,
 'pitch_body_dot_dot': 6,
 'yaw_body_dot_dot': 7,
 'roll_body_dot_dot': 8,
 'frames': 9,
 'time': 10,
 'CM_real_x_body': 11,
 'CM_real_y_body': 12,
 'CM_real_z_body': 13,
 'phi_rw_max_idx': 14,
 'phi_lw_max_idx': 15,
 'phi_rw_min_idx': 16,
 'phi_lw_min_idx': 17,
 'CM_xy': 18,
 'CM_xy_dot': 19,
 'CM_xy_dot_dot': 20,
 'yaw_z_frame': 21,
 'pitch_y_frame': 22,
 'roll_x_frame': 23,
 'yaw_z_frame_dot': 24,
 'pitch_y_frame_dot': 25,
 'roll_x_frame_dot': 26,
 'yaw_z_frame_dot_dot': 27,
 'pitch_y_frame_dot_dot': 28,
 'roll_x_frame_dot_dot': 29,
 'mean_idx': 30}

In [78]:
mov = [1,11,12,13,14,17,20,21,22,24,25,3,30,32,35,37,38,39,4,41,43,44,48,5,52,54,6,61,66,67,68,7,70,72,73,76,8]
mov_name = [f'mov{mov}' for mov in mov]
mov = [1,10,11,13,15,16,17,2,4,5,6,7,8,9]
mov_name2 = [f'mov{mov}' for mov in mov]


fig = experiment.plot_movies('time','pitch_y_frame_dot', case = 'plot_exp', fig = False, mov = mov_name)
fig.show()

fig = experiment.plot_movies('time','pitch_body', case = 'plot_exp', fig = False, mov = mov_name)
fig.show()


UnboundLocalError: local variable 'color' referenced before assignment

In [25]:

fig = experiment.plot_movies('time','CM_xy_dot', case = 'plot_exp', fig = False, mov = mov_name)
fig = experiment2.plot_movies('time','CM_xy_dot', case = 'plot_exp', fig = fig, mov = mov_name2,color = 3)
fig.show()


In [22]:
mov_name = False
mov = [1,10,11,13,15,16,17,2,4,5,6,7,8,9]
mov_name = [f'mov{mov}' for mov in mov]

fig = experiment2.plot_movies('time','pitch_y_frame', case = 'plot_mov', fig = False, mov = mov_name)
fig.show()
fig = experiment2.plot_movies('time','pitch_body', case = 'plot_mov', fig = False, mov = mov_name)
fig.show()

In [10]:

fig = experiment.plot_movies('time','pitch_y_frame', case = 'plot_mov', fig = False, mov = mov_name)
fig.show()
fig = experiment.plot_movies('time','pitch_body', case = 'plot_mov', fig = False, mov = mov_name)
fig.show()


fig = experiment.plot_movies('time','roll_x_frame', case = 'plot_mov', fig = False, mov = mov_name)
fig.show()
fig = experiment.plot_movies('time','roll_body', case = 'plot_mov', fig = False, mov = mov_name)
fig.show()

fig = experiment.plot_movies('time','yaw_z_frame', case = 'plot_mov', fig = False, mov = mov_name)
fig.show()
fig = experiment.plot_movies('time','yaw_body', case = 'plot_mov', fig = False, mov = mov_name)
fig.show()

In [ ]:
experiment.exp_dict['mov1'].data['body'][]

In [6]:

fig = experiment.plot_movies('time','pitch_y_frame', case = 'plot_mov', fig = False, mov = False)
fig.show()


fig = experiment.plot_movies('time','pitch_y_frame_dot', case = 'plot_mov', fig = False, mov = False)
fig.show()

In [12]:

fig = experiment.plot_movies('time','yaw_z_frame', case = 'plot_mov', fig = False, mov = False)
fig.show()
fig = experiment.plot_movies('time','pitch_y_frame', case = 'plot_mov', fig = False, mov = False)
fig.show()
fig = experiment.plot_movies('time','roll_x_frame', case = 'plot_mov', fig = False, mov = False)
fig.show()

In [6]:

fig = experiment.plot_movies('time','CM_xy_dot_dot', case = 'plot_mov', fig = False, mov = False)
fig.show()

In [32]:
experiment.exp_dict[mov].data['body'][:,25] - experiment.exp_dict[mov].data['body'][:,25][experiment.exp_dict[mov].ref_frame]

array([-0.01120494108875665, -0.01091411000928482, -0.01062909751261977,
       ..., 0.034560426105704624, 0.03410571371421179,
       0.03364042511486203], dtype=object)

In [19]:
 [['yaw_z_frame','pitch_y_frame','roll_x_frame'] * [deriv] for deriv in ['','_dot','_dot_dot']]

TypeError: can't multiply sequence by non-int of type 'list'

In [15]:
import numpy as np
from scipy.signal import argrelextrema, savgol_filter,find_peaks

idx = experiment.exp_dict['mov1'].header['body']['pitch_y_frame']
for mov in experiment.exp_dict.keys():

   experiment.exp_dict[mov].data['body']=np.hstack((experiment.exp_dict[mov].data['body'], (experiment.exp_dict[mov].data['body'][:,25] - experiment.exp_dict[mov].data['body'][experiment.exp_dict[mov].ref_frame,25])[np.newaxis,:].T))
   experiment.exp_dict[mov].header['body']['cm_test_dot'] = experiment.exp_dict[mov].data['body'].shape[1] - 1
fig = experiment.plot_movies('time','cm_test_dot', case = 'plot_mov', fig = False, mov = False)
fig.show()

IndexError: index 25 is out of bounds for axis 1 with size 24